In [2]:
import pandas as pd
netflix = pd.read_csv("data/processed/netflix_clean.csv")
omdb = pd.read_csv("data/processed/omdb_from_netflix.csv")
merged = netflix.merge(omdb, on="imdb_id", how="inner")
merged.head()
print(merged.shape)

(200, 35)


In [3]:
merged.columns

Index(['index', 'id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'imdb_id', 'imdb_score', 'imdb_votes',
       'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website'],
      dtype='object')

In [4]:

import pandas as pd
import numpy as np
import re

# CLEAN DATA FOR MERGED NETFLIX + OMDb DATAFRAME

merged.columns = merged.columns.str.strip()

# Drop duplicate IMDb IDs
merged = merged.drop_duplicates(subset="imdb_id", keep="first")

# Runtime Cleaning

def extract_minutes(x):
    if pd.isna(x):
        return np.nan
    matches = re.findall(r"\d+", str(x))
    return int(matches[0]) if matches else np.nan

merged["omdb_runtime_min"] = merged["Runtime"].apply(extract_minutes)

# Combine Netflix, OMDb runtime and prefer OMDb
merged["runtime_clean"] = merged["omdb_runtime_min"].fillna(merged["runtime"])


# IMDb ratings + votes cleaning

# Clean imdbVotes
if "imdbVotes" in merged.columns:
    merged["imdbVotes"] = (
        merged["imdbVotes"]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace("nan", np.nan)
        .astype(float)
    )

# Combine imdbRating from OMDb with imdb_score Netflix set
merged["imdb_rating_clean"] = (
    pd.to_numeric(merged.get("imdbRating"), errors="coerce")
        .fillna(merged.get("imdb_score"))
)
#Title cleaning
merged["title_clean"] = merged["title"].astype(str).str.lower().str.strip()
merged["Title"] = merged["Title"].astype(str).str.lower().str.strip()


merged["title_unified"] = merged["title_clean"].fillna(merged["Title"])

# Genre cleaning

merged["Genre"] = merged["Genre"].astype(str).str.split(", ")

# Clean release year

merged["release_year"] = pd.to_numeric(merged["release_year"], errors="coerce")
merged["Year"] = pd.to_numeric(merged["Year"], errors="coerce")

merged["release_year_clean"] = merged["Year"].fillna(merged["release_year"])
# ----------------------------------------------------
# Final column selection

cols_keep = [
    "id", "title_unified", "type", "release_year_clean", "age_certification",
    "runtime_clean", "Genre", "Language", "Country", "Awards", "BoxOffice",
    "Director", "Writer", "Actors", "Plot",
    "imdb_rating_clean", "imdbVotes", "imdb_id"
]

clean = merged[cols_keep]

# Rename standardized fields
clean = clean.rename(columns={
    "title_unified": "title",
    "release_year_clean": "release_year",
    "runtime_clean": "runtime_min",
    "imdb_rating_clean": "imdb_rating"
})

#Clean Df
print("CLEAN SHAPE:", clean.shape)
clean.head()


CLEAN SHAPE: (200, 18)


,id,title,type,release_year,age_certification,runtime_min,Genre,Language,Country,Awards,BoxOffice,Director,Writer,Actors,Plot,imdb_rating,imdbVotes,imdb_id
0,tm84618,taxi driver,MOVIE,1976,R,114.0,"[Crime, Drama]","English, Spanish",United States,Nominated for 4 Oscars. 22 wins & 21 nominatio...,"$28,262,574",Martin Scorsese,Paul Schrader,"Robert De Niro, Jodie Foster, Cybill Shepherd",A mentally unstable veteran works as a nightti...,8.2,1000221.0,tt0075314
1,tm127384,monty python and the holy grail,MOVIE,1975,PG,91.0,"[Adventure, Comedy, Fantasy]","English, French, Latin, Swedish",United Kingdom,2 wins & 2 nominations total,"$2,562,392","Terry Gilliam, Terry Jones","Graham Chapman, John Cleese, Eric Idle","Graham Chapman, John Cleese, Eric Idle",King Arthur and his Knights of the Round Table...,8.2,596420.0,tt0071853
2,tm70993,life of brian,MOVIE,1979,R,94.0,[Comedy],"English, Latin",United Kingdom,NaN,"$20,206,622",Terry Jones,"Graham Chapman, John Cleese, Terry Gilliam","Graham Chapman, John Cleese, Michael Palin",Born on the original Christmas in the stable n...,8.0,431314.0,tt0079470
3,tm190788,the exorcist,MOVIE,1973,R,122.0,[Horror],"English, Latin, Greek, French, German, Arabic,...",United States,Won 2 Oscars. 18 wins & 21 nominations total,"$233,005,644",William Friedkin,William Peter Blatty,"Ellen Burstyn, Max von Sydow, Linda Blair",When a mysterious entity possesses a young gir...,8.1,486836.0,tt0070047
4,tm14873,dirty harry,MOVIE,1971,R,102.0,"[Action, Crime, Thriller]",English,United States,2 wins & 4 nominations total,"$35,988,495","Don Siegel, Clint Eastwood","Harry Julian Fink, Rita M. Fink, Dean Riesner","Clint Eastwood, Andrew Robinson, Harry Guardino","When a man calling himself ""the Scorpio Killer...",7.7,177905.0,tt0066999


In [5]:
unique_genres = (
    merged["genre"]
        .astype(str)
        .str.split(",")
        .explode()
        .str.strip()
        .unique()
)

unique_genres = sorted(unique_genres)
unique_genres


KeyError: 'genre'

In [6]:

def extract_minutes(x):
    if pd.isna(x):
        return np.nan
    m = re.findall(r"\d+", str(x))
    return int(m[0]) if m else np.nan

merged["omdb_runtime_min"] = merged["Runtime"].apply(extract_minutes)


merged["runtime_min"] = merged["omdb_runtime_min"].fillna(merged["runtime"])

null_rates = merged[["Awards", "Metascore"]].isna().mean()
print("Null Rates:")
print(null_rates)

#cleaned dataset
clean = merged[
    [
        "id", "title_unified", "type", "release_year_clean",
        "age_certification", "runtime_min", "Genre", "Language",
        "Country", "Awards", "BoxOffice", "Director", "Writer",
        "Actors", "Plot"
    ]
]

clean.head()


Null Rates:
Awards       0.295
Metascore    0.520
dtype: float64


,id,title_unified,type,release_year_clean,age_certification,runtime_min,Genre,Language,Country,Awards,BoxOffice,Director,Writer,Actors,Plot
0,tm84618,taxi driver,MOVIE,1976,R,114.0,"[Crime, Drama]","English, Spanish",United States,Nominated for 4 Oscars. 22 wins & 21 nominatio...,"$28,262,574",Martin Scorsese,Paul Schrader,"Robert De Niro, Jodie Foster, Cybill Shepherd",A mentally unstable veteran works as a nightti...
1,tm127384,monty python and the holy grail,MOVIE,1975,PG,91.0,"[Adventure, Comedy, Fantasy]","English, French, Latin, Swedish",United Kingdom,2 wins & 2 nominations total,"$2,562,392","Terry Gilliam, Terry Jones","Graham Chapman, John Cleese, Eric Idle","Graham Chapman, John Cleese, Eric Idle",King Arthur and his Knights of the Round Table...
2,tm70993,life of brian,MOVIE,1979,R,94.0,[Comedy],"English, Latin",United Kingdom,NaN,"$20,206,622",Terry Jones,"Graham Chapman, John Cleese, Terry Gilliam","Graham Chapman, John Cleese, Michael Palin",Born on the original Christmas in the stable n...
3,tm190788,the exorcist,MOVIE,1973,R,122.0,[Horror],"English, Latin, Greek, French, German, Arabic,...",United States,Won 2 Oscars. 18 wins & 21 nominations total,"$233,005,644",William Friedkin,William Peter Blatty,"Ellen Burstyn, Max von Sydow, Linda Blair",When a mysterious entity possesses a young gir...
4,tm14873,dirty harry,MOVIE,1971,R,102.0,"[Action, Crime, Thriller]",English,United States,2 wins & 4 nominations total,"$35,988,495","Don Siegel, Clint Eastwood","Harry Julian Fink, Rita M. Fink, Dean Riesner","Clint Eastwood, Andrew Robinson, Harry Guardino","When a man calling himself ""the Scorpio Killer..."
